In [1]:
!pip install gym
!pip install comet_ml
!pip install box2d-py
!pip install gym[Box_2D]

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from comet_ml import Experiment
import tensorflow as tf
#tf.enable_eager_execution()


In [3]:

from mcts import Node, run_mcts
from gamecomponents import Policy
from game import Game
from replaybuffer import ReplayBuffer
from network import Network, SharedStorage
import random
import numpy as np
from helpers import KnownBounds
from muzero_core import play_game, train_network, MuZeroConfig, make_aigym_config

In [4]:
c = make_aigym_config('CartPole-v0')
s = SharedStorage()
r = ReplayBuffer()
#r.window_size = 90
    
c.num_simulations = 60
c.training_steps = 150
c.batch_size = 32
c.td_steps = 25
c.discount = 1. #[0.999, 0.9]
c.lr_init = 0.02
c.lr_decay_steps = 100e3
c.max_moves = 2000
#c.visit_softmax_temperature = partial(visit_softmax_temperature
c.known_bounds = KnownBounds(max = 20, min = -20)
print(c)
c.epsilon = 0.001

Network.N = 6
Network.grad_scale = (1.,1.,1.)
#Node.root_exploration_fraction = 0.30
#Node.root_dirichlet_alpha = 0.35
#r.window_size = 25
for _ in range(30):
  n = s.latest_network()
  g1 = play_game(c, n)
  r.save_game(g1)
  print(sum(g1.rewards))

19.0
46.0
25.0
20.0
33.0
34.0
9.0
12.0
10.0
23.0
24.0
11.0
23.0
13.0
37.0
38.0
20.0
16.0
13.0
16.0
42.0
24.0
22.0
44.0
10.0
19.0
20.0
8.0
19.0
14.0


In [ ]:
experiment = Experiment(api_key="P2vxSVfy24E3snd418Cmx4Nw2",
                      project_name="muzero-cartpoleNstep", workspace="pnorridge")

from statistics import mean 

N = 10
st = 0
for inter in range(400):

  train_network(c,s,r,experiment)

  n = s.latest_network()
  #r = ReplayBuffer()
  tot = 0
  N = 0
  better = True
  while better:
    g1 = play_game(c, n)
    print(g1.length())
    print(g1.root_values[0])
    tot += (g1.length())
    N += 1
    better = tot/N > max(r.game_len)*0.5 and N < 25
    r.save_game(g1)
    
  print((inter, tot/N))
  experiment.log_metric("count", tot/N, step=n.training_steps())
  experiment.log_metric("replay_buffer_size", len(r.buffer), step=n.training_steps())
  print('----')


    
experiment.end()



COMET INFO: Experiment is live on comet.ml https://www.comet.ml/pnorridge/muzero-cartpolenstep/5a6c83aea0704b8f9ff552d35811d720



Instructions for updating:
Use tf.identity instead.
(0, (12.796543, 12.737388, 1.3862944))
(100, (2.6498737, 7.0753303, 1.3858738))
60
[16.340406]
16
[16.306362]
11
[16.260435]
(0, 29.0)
----
(0, (3.6446798, 7.075293, 1.3845377))
(100, (2.4362943, 7.07529, 1.3856039))
12
[17.32642]
(1, 12.0)
----
(0, (2.5578785, 7.0752883, 1.3877115))
(100, (2.7554982, 7.075288, 1.387814))
56
[17.777418]
29
[17.777472]
13
[17.777435]
16
[17.768341]
(2, 28.5)
----
(0, (2.689096, 7.075289, 1.3864532))
(100, (2.3906844, 7.075288, 1.3867577))
18
[16.477818]
(3, 18.0)
----
(0, (2.4781284, 7.0752873, 1.3864026))
(100, (2.2542973, 7.075289, 1.3853787))
35
[17.708488]
17
[17.683338]
(4, 26.0)
----
(0, (2.565545, 7.075289, 1.3871074))
(100, (2.3738766, 7.0752916, 1.3851656))
41
[16.856922]
15
[16.856495]
(5, 28.0)
----
(0, (2.5244129, 7.075295, 1.3857129))
(100, (2.9853764, 7.075298, 1.3869172))
11
[17.67877]
(6, 11.0)
----
(0, (2.3591466, 7.075299, 1.3888488))
(100, (2.9110422, 7.0753136, 1.3847994))
19
[18.30

In [ ]:
r.sample_position(r.buffer[0])

In [ ]:
c = make_aigym_config('LunarLander-v2')
s = SharedStorage()
r = ReplayBuffer()
r.window_size = 90
    
c.num_simulations = 60 
c.training_steps = 300
c.batch_size = 32
c.td_steps = 25
c.discount = 0.999
c.lr_init = 0.03
c.lr_decay_steps = 100e3
c.max_moves = 300
#c.known_bounds = KnownBounds(max = 20, min = -20)
print(c)

Network.N = 8
Node.root_exploration_fraction = 0.50
Node.root_dirichlet_alpha = 0.35
#r.window_size = 25
for _ in range(1):
  n = s.latest_network()
  g1 = play_game(c, n)
  r.save_game(g1)
  print(sum(g1.rewards))

In [ ]:
experiment = Experiment(api_key="P2vxSVfy24E3snd418Cmx4Nw2",
                      project_name="muzero", workspace="pnorridge")

from statistics import mean 

N = 10
st = 0
for inter in range(300):

  train_network(c,s,r,experiment)

  n = s.latest_network()
  #r = ReplayBuffer()
  tot = 0
  N = 0
  #better = True
  #while better:
  g1 = play_game(c, n)
  #better = N < 3
  r.save_game(g1)
  print(sum(g1.rewards))
  print(g1.root_values[0])
  tot += (sum(g1.rewards))
  N += 1

    
  print((inter, tot/N))
  experiment.log_metric("count", tot/N, step=n.training_steps())
  experiment.log_metric("replay_buffer_size", len(r.buffer), step=n.training_steps())
  print('----')


    
experiment.end()



COMET INFO: Experiment is live on comet.ml https://www.comet.ml/pnorridge/muzero/964f795fab5946e68d964b9616a4e237



Instructions for updating:
Use tf.identity instead.
(0, (12.845146, 12.810631, 2.7834191))
(100, (2.967114, 7.5585814, 2.3096893))
(200, (1.7222418, 7.280098, 2.311709))
(300, (1.8253598, 7.2430873, 2.411283))
(300, (3.3480582, 7.3783402, 1.9712055))
-309.06161946130976
-83.091415
(1, -309.06161946130976)
----
(0, (5.3563185, 8.125771, 2.1759858))
(100, (4.1303353, 7.920557, 2.236796))
(200, (3.3177922, 7.5306883, 2.014202))
(300, (3.082624, 7.65125, 2.4820793))
-141.1912872655052
-26.338614
(2, -141.1912872655052)
----
(0, (3.5253696, 7.6220937, 2.2522295))
(100, (3.8799567, 7.4551334, 1.9804547))
(200, (2.96663, 7.2893386, 2.0931094))
(300, (2.4052076, 7.2591863, 1.9694053))
-292.3744830238216
-25.81643
(3, -292.3744830238216)
----
(0, (6.2501144, 7.9013495, 2.6591108))
(100, (3.4417934, 7.554871, 2.329184))
(200, (3.2930923, 7.321356, 2.5089698))
(300, (3.0247884, 7.357904, 2.4813368))
-126.31768719821912
-113.25505
(4, -126.31768719821912)
----
(0, (3.8171763, 7.6487274, 2.5310035)

In [5]:
x = []
x

[]

In [8]:
x = [np.array([1.]),np.array(1.)]

In [9]:
x

[array([1.]), array(1.)]

In [10]:
list(x)

[array([1.]), array(1.)]